In [1]:
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import mean_squared_error, r2_score

# Load the TSV file
df = pd.read_csv('ml-32m/ratings.csv')


# Print the head of the DataFrame
print(df.head())

/Users/alexanderaujesky/opt/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-11-15 10:33:03.638792: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


   userId  movieId  rating            timestamp
0       1        2     3.5  2005-04-02 23:53:47
1       1       29     3.5  2005-04-02 23:31:16
2       1       32     3.5  2005-04-02 23:33:39
3       1       47     3.5  2005-04-02 23:32:07
4       1       50     3.5  2005-04-02 23:29:40


In [2]:
data_folder = './MovieSummaries/MovieSummaries/'
#paths to files
plot_summaries_path = data_folder + 'plot_summaries.txt'
movie_metadata_path = data_folder + 'movie.metadata.tsv'
character_metadata_path = data_folder + 'character.metadata.tsv'

# load the data
# 1. Plot summaries data
plot_summaries_df = pd.read_csv(plot_summaries_path, delimiter='\t', names=['wikipedia_movie_id', 'plot_summary'], 
                                 encoding='utf-8')

# 2. Movie metadata
movie_metadata_df = pd.read_csv(movie_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 
                                                                            'movie_name', 'release_date', 'box_office_revenue',
                                                                            'runtime', 'languages', 'countries', 'genres'], 
                                 encoding='utf-8')

# 3. Character metadata
character_metadata_df = pd.read_csv(character_metadata_path, delimiter='\t', names=['wikipedia_movie_id', 'freebase_movie_id', 'release_date', 'character_name', 
                                                                                    'actor_dob', 'actor_gender', 'actor_height', 'actor_ethnicity', 'actor_name', 
                                                                                    'actor_age_at_release', 'freebase_character_actor_map_id', 'freebase_character_id', 
                                                                                    'freebase_actor_id'], 
                                     encoding='utf-8')


In [3]:
movies = pd.read_csv('ml-32m/movies.csv')

In [4]:
movie_metadata_df['movie_name_formatted'] = movie_metadata_df['movie_name'].str.lower().str.strip()
movies['title_format'] = movies['title'].str[:-6].str.strip().str.lower()


In [5]:
movies.head()

,movieId,title,genres,title_format
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii


In [6]:
# calculate the number of common movies between the two datasets

common_movies = set(movie_metadata_df['movie_name_formatted']).intersection(set(movies['title_format']))
print('Number of common movies:', len(common_movies))

# merge the two datasets

merged_df = pd.merge(movies, movie_metadata_df, left_on='title_format', right_on='movie_name_formatted', how='inner')

Number of common movies: 12318


In [7]:
merged_df

,movieId,title,genres_x,title_format,wikipedia_movie_id,freebase_movie_id,movie_name,release_date,box_office_revenue,runtime,languages,countries,genres_y,movie_name_formatted
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,53085,/m/0dyb1,Toy Story,1995-11-19,361958736.0,77.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0556j8"": ""Buddy film"", ""/m/03k9fj"": ""Adve...",toy story
1,2,Jumanji (1995),Adventure|Children|Fantasy,jumanji,3700174,/m/09w353,Jumanji,1995-12-15,262797249.0,104.0,"{""/m/064_8sq"": ""French Language"", ""/m/02h40lc""...","{""/m/09c7w0"": ""United States of America"", ""/m/...","{""/m/01jfsb"": ""Thriller"", ""/m/0hj3n2k"": ""Fanta...",jumanji
2,3,Grumpier Old Men (1995),Comedy|Romance,grumpier old men,1934035,/m/0676dr,Grumpier Old Men,1995-12-22,71518503.0,101.0,"{""/m/02bjrlw"": ""Italian Language"", ""/m/02h40lc...","{""/m/09c7w0"": ""United States of America""}","{""/m/06cvj"": ""Romantic comedy"", ""/m/0556j8"": ""...",grumpier old men
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,waiting to exhale,972970,/m/03vny7,Waiting to Exhale,1995-12-22,81452156.0,121.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hj3n0w"": ""Ensemble Film"", ""/m/06w2n3t"": ...",waiting to exhale
4,5,Father of the Bride Part II (1995),Comedy,father of the bride part ii,3303622,/m/094g2z,Father of the Bride Part II,1995-12-08,76594107.0,106.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/06qm3"": ""Screwball comedy"", ""/m/02l7c8"": ...",father of the bride part ii
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17557,131152,The Fat Spy (1966),Comedy,the fat spy,1853200,/m/0613sx,The Fat Spy,1966,NaN,75.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0gf28"": ""Parody"", ""/m/0220p9g"": ""Musical ...",the fat spy
17558,131166,WWII IN HD (2009),(no genres listed),wwii in hd,26321497,/m/0bbxzy2,WWII in HD,NaN,NaN,NaN,{},{},"{""/m/082gq"": ""War film""}",wwii in hd
17559,131239,Three Quarter Moon (2011),Comedy|Drama,three quarter moon,33597212,/m/0hgrd5g,Three Quarter Moon,2011-09-30,NaN,95.0,"{""/m/02hwyss"": ""Turkish Language"", ""/m/0gtg"": ...","{""/m/0345h"": ""Germany""}","{""/m/0556j8"": ""Buddy film"", ""/m/0hqxf"": ""Famil...",three quarter moon
17560,131248,Brother Bear 2 (2006),Adventure|Animation|Children|Comedy|Fantasy,brother bear 2,4118248,/m/0bk2k2,Brother Bear 2,2006-08-29,NaN,73.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/0hqxf"": ""Family Film"", ""/m/0hcr"": ""Animat...",brother bear 2


In [8]:
# Remove this user from the dataset, because it has too many ratings
df = df[df['userId'] != 175325]

In [9]:
list_of_genres=movies['genres'].unique()

In [10]:
# Print the user with the most ratings
user_ratings_count = df['userId'].value_counts()

print('User with the most ratings:')
print(user_ratings_count.head(20))


User with the most ratings:
118205    9254
8405      7515
82418     5646
121535    5520
125794    5491
74142     5447
34576     5356
131904    5330
83090     5169
59477     4988
130767    4785
79159     4707
8963      4524
15617     4354
92011     4236
71975     4182
20132     4101
46470     4094
88820     4093
63147     3958
Name: userId, dtype: int64


In [11]:
rating_films=pd.merge(movies,df,on='movieId',how='inner')


In [12]:
unique_genres = movies['genres'].unique()
all_genres = set('|'.join(unique_genres).split('|'))

all_genres_list = list(all_genres)
all_genres_list.remove('(no genres listed)')
print(all_genres_list)


['Thriller', 'Film-Noir', 'War', 'Animation', 'IMAX', 'Fantasy', 'Drama', 'Mystery', 'Horror', 'Sci-Fi', 'Adventure', 'Western', 'Children', 'Comedy', 'Romance', 'Crime', 'Musical', 'Action', 'Documentary']


In [13]:
# Create a one hot encoding for each genre
for genre in all_genres_list : 
    rating_films[genre] = rating_films['genres'].apply(lambda x: genre in x.split('|'))
rating_films

,movieId,title,genres,title_format,userId,rating,timestamp,Thriller,Film-Noir,War,...,Sci-Fi,Adventure,Western,Children,Comedy,Romance,Crime,Musical,Action,Documentary
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,3,4.0,1999-12-11 13:36:47,False,False,False,...,False,True,False,True,True,False,False,False,False,False
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,6,5.0,1997-03-13 17:50:52,False,False,False,...,False,True,False,True,True,False,False,False,False,False
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,8,4.0,1996-06-05 13:37:51,False,False,False,...,False,True,False,True,True,False,False,False,False,False
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,10,4.0,1999-11-25 02:44:47,False,False,False,...,False,True,False,True,True,False,False,False,False,False
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,toy story,11,4.5,2009-01-02 01:13:41,False,False,False,...,False,True,False,True,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20000258,131254,Kein Bund für's Leben (2007),Comedy,kein bund für's leben,79570,4.0,2015-03-30 19:32:59,False,False,False,...,False,False,False,False,True,False,False,False,False,False
20000259,131256,"Feuer, Eis & Dosenbier (2002)",Comedy,"feuer, eis & dosenbier",79570,4.0,2015-03-30 19:48:08,False,False,False,...,False,False,False,False,True,False,False,False,False,False
20000260,131258,The Pirates (2014),Adventure,the pirates,28906,2.5,2015-03-30 19:56:32,False,False,False,...,False,True,False,False,False,False,False,False,False,False
20000261,131260,Rentun Ruusu (2001),(no genres listed),rentun ruusu,65409,3.0,2015-03-30 19:57:46,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [14]:
def rating_for_userID_generator(userId):
    user_rating = rating_films[rating_films['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])
    
    length = len(y)
    x_size = int(length * 0.8) 
    
    x_train = x.iloc[:x_size, :]
    x_test = x.iloc[x_size:, :]
    y_train = y.iloc[:x_size]
    y_test = y.iloc[x_size:]
    
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = rating_for_userID_generator(8405)


In [15]:


# Define the neural network model
model = Sequential([
    Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
    Dense(32, activation='relu'),  # Second hidden layer
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mse'])

# Train the model
history = model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Predict on both training and test sets
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Evaluate the model
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)

print("Training Set Evaluation:")
print("MSE:", train_mse)
print("R^2 Score:", train_r2)

print("\nTest Set Evaluation:")
print("MSE:", test_mse)
print("R^2 Score:", test_r2)



Epoch 1/100
151/151 [==============================] - 1s 4ms/step - loss: 2.7784 - mse: 2.7784 - val_loss: 0.9295 - val_mse: 0.9295
Epoch 2/100
151/151 [==============================] - 1s 3ms/step - loss: 0.8512 - mse: 0.8512 - val_loss: 0.8112 - val_mse: 0.8112
Epoch 3/100
151/151 [==============================] - 1s 3ms/step - loss: 0.7950 - mse: 0.7950 - val_loss: 0.7861 - val_mse: 0.7861
Epoch 4/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7792 - mse: 0.7792 - val_loss: 0.7809 - val_mse: 0.7809
Epoch 5/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7737 - mse: 0.7737 - val_loss: 0.7802 - val_mse: 0.7802
Epoch 6/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7698 - mse: 0.7698 - val_loss: 0.7821 - val_mse: 0.7821
Epoch 7/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7635 - mse: 0.7635 - val_loss: 0.7769 - val_mse: 0.7769
Epoch 8/100
151/151 [==============================] - 0s 2ms/step - 

In [17]:

def evaluate_model_for_user(userId, data):
    """
    Generates training and test splits for a given user ID, trains a neural network, 
    and evaluates the model to return MSE and R^2 for both training and test sets.
    """
    # Filter and preprocess data for the given user
    user_rating = data[data['userId'] == userId]
    user_rating = user_rating.iloc[:, 5:].sample(frac=1)  # Shuffle data
    user_rating = user_rating.drop(columns=['timestamp'])
    y = user_rating['rating']
    x = user_rating.drop(columns=['rating'])

    # Split into training and testing sets
    length = len(y)
    x_size = int(length * 0.8)
    x_train, x_test = x.iloc[:x_size, :], x.iloc[x_size:, :]
    y_train, y_test = y.iloc[:x_size], y.iloc[x_size:]

    # Define the neural network model
    model = Sequential([
        Dense(64, input_dim=x_train.shape[1], activation='relu'),  # First hidden layer
        Dense(32, activation='relu'),  # Second hidden layer
        Dense(1, activation='linear')  # Output layer for regression
    ])

    # Compile the model
    model.compile(optimizer='adam', loss='mse', metrics=['mse'])

    # Train the model
    model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

    # Predict on training and test sets
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    # Calculate evaluation metrics
    train_mse = mean_squared_error(y_train, y_train_pred)
    test_mse = mean_squared_error(y_test, y_test_pred)
    train_r2 = r2_score(y_train, y_train_pred)
    test_r2 = r2_score(y_test, y_test_pred)

    # Print evaluation results
    print(f"Evaluation for User ID {userId}:")
    print("Training Set Evaluation:")
    print(f"MSE: {train_mse:.4f}, R^2: {train_r2:.4f}")
    print("\nTest Set Evaluation:")
    print(f"MSE: {test_mse:.4f}, R^2: {test_r2:.4f}")

    # Return MSE values for both sets
    return train_mse, test_mse

train_mse, test_mse = evaluate_model_for_user(8405, rating_films)


Epoch 1/100
151/151 [==============================] - 1s 3ms/step - loss: 3.3125 - mse: 3.3125 - val_loss: 0.8634 - val_mse: 0.8634
Epoch 2/100
151/151 [==============================] - 0s 2ms/step - loss: 0.8529 - mse: 0.8529 - val_loss: 0.7774 - val_mse: 0.7774
Epoch 3/100
151/151 [==============================] - 0s 2ms/step - loss: 0.7978 - mse: 0.7978 - val_loss: 0.7391 - val_mse: 0.7391
Epoch 4/100
151/151 [==============================] - 0s 3ms/step - loss: 0.7780 - mse: 0.7780 - val_loss: 0.7344 - val_mse: 0.7344
Epoch 5/100
151/151 [==============================] - 0s 3ms/step - loss: 0.7735 - mse: 0.7735 - val_loss: 0.7257 - val_mse: 0.7257
Epoch 6/100
151/151 [==============================] - 1s 5ms/step - loss: 0.7646 - mse: 0.7646 - val_loss: 0.7233 - val_mse: 0.7233
Epoch 7/100
151/151 [==============================] - 1s 3ms/step - loss: 0.7606 - mse: 0.7606 - val_loss: 0.7376 - val_mse: 0.7376
Epoch 8/100
151/151 [==============================] - 1s 6ms/step - 